# Eval Metrics

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.models.loader import DataModule

In [10]:
dm = DataModule()
dm.val_data

HeteroData(
  recipe={
    x=[231637, 9],
    id=[231637]
  },
  user={
    x=[226570, 0],
    id=[226570]
  },
  (user, reviews, recipe)={
    edge_index=[2, 905895],
    edge_attr=[905895],
    edge_label=[113236],
    edge_label_index=[2, 113236]
  },
  (recipe, rev_reviews, user)={
    edge_index=[2, 905895],
    edge_attr=[905895]
  }
)

In [17]:
x = next(iter(dm.val_dataloader()))
x

HeteroData(
  recipe={
    x=[181785, 9],
    id=[181785]
  },
  user={
    x=[148937, 0],
    id=[148937]
  },
  (user, reviews, recipe)={
    edge_index=[2, 751213],
    edge_attr=[751213],
    edge_label_index=[2, 2048],
    input_id=[2048]
  },
  (recipe, rev_reviews, user)={
    edge_index=[2, 668586],
    edge_attr=[668586]
  }
)

In [19]:
x['reviews'].edge_label_index

tensor([[1434, 1618,   76,  ...,  675,  200, 1477],
        [1250,  762,  170,  ...,  702,  604, 1778]])